In [1]:
import keras_forward as kf
import tf_forward as tff
import keras_models as kms
import tf_graphs as tfgs
import numpy as np

/home/astrophysics/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
num_inputs = 28 * 28
num_outputs = 10
m = 1000
batch_size = 1000
a1_size = 5

km = kms.slp_model(num_inputs, num_outputs)
km.compile(loss='mse', optimizer='rmsprop')
np.random.seed(1337)
x_tr = np.random.random((m, num_inputs))
y_tr = np.random.randint(size=(m, num_outputs), low = 0, high = 2)
num_weights = (num_inputs + 1) * a1_size + (a1_size + 1) * num_outputs

km = kf.keras_model(km, x_tr, y_tr, batch_size) 
km.setup_LL()

tf_graph = tfgs.slp_graph
layer_sizes = [a1_size]
tfm = tff.tf_model(tf_graph, x_tr, y_tr, batch_size, layer_sizes)
fit_metric = 'chisq'
tfm.setup_LL(fit_metric)

n = 1
weights = np.random.random((n, num_weights))


Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [3]:
%%timeit
for i in range(n):
    tfm(weights[i])

10 loops, best of 3: 21 ms per loop


In [4]:
%%timeit
for i in range(n):
    km(weights[i])

10 loops, best of 3: 41.3 ms per loop


In [5]:
import pstats, cProfile

In [5]:
cProfile.runctx("km(w)", globals(), locals(), "kmProfile.prof")

s = pstats.Stats("kmProfile.prof")
s.strip_dirs().sort_stats("time").print_stats()


3/3 [==============================]3/3 [==============================] - 0s 1ms/step

Wed Sep 19 11:43:38 2018    kmProfile.prof

         1510 function calls (1504 primitive calls) in 0.017 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       65    0.007    0.000    0.007    0.000 {id}
        1    0.001    0.001    0.001    0.001 {time.sleep}
        2    0.001    0.001    0.001    0.001 {_pywrap_tensorflow_internal.TF_Run}
      224    0.000    0.000    0.000    0.000 {isinstance}
        2    0.000    0.000    0.012    0.006 session.py:1042(_run)
        2    0.000    0.000    0.001    0.000 session.py:340(__init__)
       10    0.000    0.000    0.000    0.000 socket.py:333(send)
       19    0.000    0.000    0.000    0.000 ops.py:322(name)
       65    0.000    0.000    0.007    0.000 ops.py:564(__hash__)
        2    0.000    0.000    0.001    0.001 session.py:407(__init__)
       46    0.000    0.000    0.000  

<pstats.Stats instance at 0x7fb49001f2d8>

In [6]:
cProfile.runctx("tfm(w)", globals(), locals(), "tfmProfile.prof")

s = pstats.Stats("tfmProfile.prof")
s.strip_dirs().sort_stats("time").print_stats()

Wed Sep 19 12:40:52 2018    tfmProfile.prof

         26632 function calls (26604 primitive calls) in 0.492 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.149    0.074    0.149    0.074 {_pywrap_tensorflow_internal.TF_Run}
        2    0.096    0.048    0.096    0.048 {_pywrap_tensorflow_internal.TF_ExtendGraph}
       31    0.036    0.001    0.039    0.001 ops.py:2741(_extract_stack)
     1391    0.023    0.000    0.023    0.000 {method 'extend' of 'google.protobuf.pyext._message.RepeatedCompositeContainer' objects}
       83    0.022    0.000    0.022    0.000 {method 'SerializeToString' of 'google.protobuf.pyext._message.CMessage' objects}
        1    0.018    0.018    0.018    0.018 {_pywrap_tensorflow_internal.TF_DeleteDeprecatedSession}
        2    0.017    0.008    0.059    0.030 ops.py:2925(_as_graph_def)
     3204    0.016    0.000    0.016    0.000 {isinstance}
     1453    0.013    0.000    0.01

        1    0.000    0.000    0.002    0.002 gen_math_ops.py:2920(_pow)
        5    0.000    0.000    0.000    0.000 tensor_shape.py:785(is_fully_defined)
        4    0.000    0.000    0.000    0.000 variables.py:740(_TensorConversionFunction)
        2    0.000    0.000    0.000    0.000 tensor_util.py:265(_FilterFloat)
       11    0.000    0.000    0.000    0.000 tensor_shape.py:787(<genexpr>)
        1    0.000    0.000    0.000    0.000 ops.py:3634(get_collection)
        1    0.000    0.000    0.000    0.000 copy.py:306(_reconstruct)
        3    0.000    0.000    0.000    0.000 tensor_util.py:161(GetFromNumpyDTypeDict)
        1    0.000    0.000    0.002    0.002 math_ops.py:530(pow)
        5    0.000    0.000    0.000    0.000 _methods.py:34(_prod)
        4    0.000    0.000    0.000    0.000 tensor_shape.py:83(is_compatible_with)
        2    0.000    0.000    0.000    0.000 device.py:65(__init__)
        2    0.000    0.000    0.000    0.000 session.py:456(targets)
    

<pstats.Stats instance at 0x7f0c5a5d3878>